# Pagsasanay ng RL para sa Pagbalanse ng Cartpole

Ang notebook na ito ay bahagi ng [AI for Beginners Curriculum](http://aka.ms/ai-beginners). Ito ay inspirasyon mula sa [opisyal na PyTorch tutorial](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) at [Cartpole PyTorch implementation na ito](https://github.com/yc930401/Actor-Critic-pytorch).

Sa halimbawang ito, gagamit tayo ng RL upang sanayin ang isang modelo na magbalanse ng poste sa isang cart na maaaring gumalaw pakaliwa at pakanan sa isang pahalang na sukat. Gagamitin natin ang [OpenAI Gym](https://www.gymlibrary.ml/) na kapaligiran upang gayahin ang poste.

> **Note**: Maaari mong patakbuhin ang code ng araling ito nang lokal (hal. mula sa Visual Studio Code), kung saan magbubukas ang simulation sa isang bagong window. Kapag pinapatakbo ang code online, maaaring kailanganin mong gumawa ng ilang pagbabago sa code, tulad ng inilarawan [dito](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7).

Magsisimula tayo sa pamamagitan ng pagtiyak na naka-install ang Gym:


In [ ]:
import sys
!{sys.executable} -m pip install gym

Ngayon, gumawa tayo ng CartPole environment at tingnan kung paano ito gamitin. Ang isang environment ay may mga sumusunod na katangian:

* **Action space** ay ang hanay ng mga posibleng aksyon na maaari nating gawin sa bawat hakbang ng simulation  
* **Observation space** ay ang hanay ng mga obserbasyon na maaari nating makita  


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

Tingnan natin kung paano gumagana ang simulation. Ang sumusunod na loop ay nagpapatakbo ng simulation hanggang sa ang `env.step` ay hindi magbalik ng termination flag na `done`. Pipili tayo ng mga aksyon nang random gamit ang `env.action_space.sample()`, na nangangahulugang malamang na mabigo agad ang eksperimento (ang CartPole environment ay nagtatapos kapag ang bilis ng CartPole, ang posisyon nito, o anggulo ay lumampas sa mga tiyak na limitasyon).

> Magbubukas ang simulation sa bagong window. Maaari mong patakbuhin ang code nang ilang beses at obserbahan kung paano ito kumikilos.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

Mapapansin mo na ang mga obserbasyon ay binubuo ng 4 na numero. Ang mga ito ay:
- Posisyon ng kariton
- Bilis ng kariton
- Anggulo ng poste
- Bilis ng pag-ikot ng poste

Ang `rew` ay ang gantimpala na natatanggap natin sa bawat hakbang. Makikita mo na sa CartPole na kapaligiran, ikaw ay ginagantimpalaan ng 1 puntos para sa bawat hakbang ng simulasyon, at ang layunin ay pataasin ang kabuuang gantimpala, ibig sabihin, ang oras na kayang balansehin ng CartPole nang hindi bumabagsak.

Sa reinforcement learning, ang layunin natin ay sanayin ang isang **patakaran** $\pi$, na para sa bawat estado $s$ ay magsasabi sa atin kung aling aksyon $a$ ang dapat gawin, kaya sa esensya $a = \pi(s)$.

Kung nais mo ng probabilistikong solusyon, maaari mong isipin ang patakaran bilang pagbibigay ng hanay ng mga probabilidad para sa bawat aksyon, ibig sabihin, ang $\pi(a|s)$ ay nangangahulugan ng probabilidad na dapat nating gawin ang aksyon $a$ sa estado $s$.

## Paraan ng Policy Gradient

Sa pinakasimpleng RL algorithm, na tinatawag na **Policy Gradient**, magsasanay tayo ng isang neural network upang mahulaan ang susunod na aksyon.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

Aating sasanayin ang network sa pamamagitan ng pagsasagawa ng maraming eksperimento, at ia-update ang network pagkatapos ng bawat takbo. Magde-define tayo ng isang function na magsasagawa ng eksperimento at magbabalik ng mga resulta (tinatawag na **trace**) - lahat ng estado, aksyon (at ang kanilang inirerekomendang mga probabilidad), at mga gantimpala:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

Maaari kang magpatakbo ng isang episode gamit ang hindi sanay na network at mapansin na ang kabuuang gantimpala (kilala rin bilang haba ng episode) ay napakababa:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

Isa sa mga mahirap na aspeto ng policy gradient algorithm ay ang paggamit ng **discounted rewards**. Ang ideya ay kinukuwenta natin ang vector ng kabuuang rewards sa bawat hakbang ng laro, at sa prosesong ito ay dinidiskwento natin ang mga maagang rewards gamit ang isang coefficient $gamma$. Ina-normalize din natin ang resulting vector, dahil gagamitin natin ito bilang timbang upang makaapekto sa ating training:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

Ngayon, magsimula na tayo sa aktwal na pagsasanay! Tatakbo tayo ng 300 episodes, at sa bawat episode ay gagawin natin ang mga sumusunod:

1. Patakbuhin ang eksperimento at kolektahin ang trace.
2. Kalkulahin ang pagkakaiba (`gradients`) sa pagitan ng mga ginawang aksyon at ng mga hinulaang probabilidad. Kapag mas maliit ang pagkakaiba, mas sigurado tayo na tama ang aksyong ginawa.
3. Kalkulahin ang mga discounted rewards at imultiply ang gradients sa mga discounted rewards - titiyakin nito na ang mga hakbang na may mas mataas na gantimpala ay magkakaroon ng mas malaking epekto sa panghuling resulta kumpara sa mga hakbang na may mas mababang gantimpala.
4. Ang mga inaasahang target na aksyon para sa ating neural network ay bahagyang manggagaling sa mga hinulaang probabilidad habang tumatakbo, at bahagyang mula sa mga kalkuladong gradients. Gagamitin natin ang parameter na `alpha` upang matukoy kung hanggang saan isasaalang-alang ang gradients at rewards - ito ay tinatawag na *learning rate* ng reinforcement algorithm.
5. Sa wakas, sasanayin natin ang ating network sa mga estado at inaasahang aksyon, at uulitin ang proseso.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

Ngayon, patakbuhin natin ang episode na may rendering upang makita ang resulta:


In [ ]:
_ = run_episode(render=True)

Sana napansin mo na ang poste ay kaya nang magbalanse nang maayos!

## Actor-Critic Model

Ang Actor-Critic model ay isang mas advanced na bersyon ng policy gradients, kung saan gumagawa tayo ng neural network upang matutunan ang parehong policy at tinatayang mga gantimpala. Ang network ay magkakaroon ng dalawang output (o maaari mo itong tingnan bilang dalawang magkahiwalay na network):
* **Actor** ang magrerekomenda ng aksyon na dapat gawin sa pamamagitan ng pagbibigay sa atin ng state probability distribution, tulad ng sa policy gradient model.
* **Critic** ang magtatantiya kung ano ang magiging gantimpala mula sa mga aksyong iyon. Ibabalik nito ang kabuuang tinatayang gantimpala sa hinaharap sa ibinigay na estado.

Tukuyin natin ang ganitong modelo:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

Kailangan nating bahagyang baguhin ang ating mga `discounted_rewards` at `run_episode` na mga function:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


Ngayon, tatakbuhin natin ang pangunahing training loop. Gagamitin natin ang manual na proseso ng pagsasanay ng network sa pamamagitan ng pagkalkula ng tamang loss functions at pag-update ng mga parameter ng network:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## Mga Mahalagang Punto

Nakita natin ang dalawang RL algorithm sa demo na ito: simple policy gradient at ang mas sopistikadong actor-critic. Makikita mo na ang mga algorithm na ito ay gumagana gamit ang mga abstraktong konsepto ng estado, aksyon, at gantimpala - kaya maaari silang i-apply sa iba't ibang uri ng kapaligiran.

Ang reinforcement learning ay nagbibigay-daan sa atin na matutunan ang pinakamainam na estratehiya para lutasin ang problema sa pamamagitan lamang ng pagtingin sa huling gantimpala. Ang katotohanan na hindi natin kailangan ng mga labelled dataset ay nagbibigay-daan sa atin na ulitin ang mga simulation nang maraming beses upang ma-optimize ang ating mga modelo. Gayunpaman, marami pa ring mga hamon sa RL, na maaari mong matutunan kung magpapasya kang magpokus nang higit pa sa kawili-wiling larangang ito ng AI.



---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, tandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na dulot ng paggamit ng pagsasaling ito.
